calculate the % marks for each student. for each subject is of 100 marks. 

create a result column by following the below condition: 
1. % marks greater than or equal to 70 then 'Distinction' 
2. % marks range between 60-69 then  'First Class' 
3. % marks range between 50-59 then 'Second Class' 
4. % marks range between 40-49 then 'Third Class' 
5. % marks less than or equal to 39 then 'Fail'

In [0]:
data1=[(1, "Steve"), (2, "David"), (3, "John"), (4, "Shree"), (5, "Helen" )] 
data2=[ (1, "SQL",90), (1, "PySpark",100), (2, "SQL",70), (2, "PySpark",60), (3, "SQL", 30), (3,"PySpark", 50), 
       (5,"SQL",45), (5, "PySpark", 45)]

schema1=["Id","Name" ]
schema2=["Id","Subject","Mark"]

df1=spark.createDataFrame(data1,schema1)
df2=spark.createDataFrame(data2,schema2)

display(df1)
display(df2)

Id,Name
1,Steve
2,David
3,John
4,Shree
5,Helen


Id,Subject,Mark
1,SQL,90
1,PySpark,100
2,SQL,70
2,PySpark,60
3,SQL,30
3,PySpark,50
5,SQL,45
5,PySpark,45


In [0]:
df_join=df1.join(df2,"id","inner")
df_join.show()

+---+-----+-------+----+
| Id| Name|Subject|Mark|
+---+-----+-------+----+
|  1|Steve|    SQL|  90|
|  1|Steve|PySpark| 100|
|  2|David|    SQL|  70|
|  2|David|PySpark|  60|
|  3| John|    SQL|  30|
|  3| John|PySpark|  50|
|  5|Helen|    SQL|  45|
|  5|Helen|PySpark|  45|
+---+-----+-------+----+



In [0]:
df_join=df1.join(df2,df1.Id==df2.Id,"inner").drop(df2.Id)
df_join.show()

+---+-----+-------+----+
| Id| Name|Subject|Mark|
+---+-----+-------+----+
|  1|Steve|    SQL|  90|
|  1|Steve|PySpark| 100|
|  2|David|    SQL|  70|
|  2|David|PySpark|  60|
|  3| John|    SQL|  30|
|  3| John|PySpark|  50|
|  5|Helen|    SQL|  45|
|  5|Helen|PySpark|  45|
+---+-----+-------+----+



In [0]:
from pyspark.sql.functions import *
df_percent=df_join.groupBy("Id","Name").agg(
    (sum("Mark")/count("*")).alias("percentage")
)

df_percent.show()


+---+-----+----------+
| Id| Name|percentage|
+---+-----+----------+
|  1|Steve|      95.0|
|  2|David|      65.0|
|  3| John|      40.0|
|  5|Helen|      45.0|
+---+-----+----------+



In [0]:
df_final = df_percent.select("*",
                  when (df_percent.percentage>=70,'Distinction')
                  .when ((df_percent.percentage<70) & (df_percent.percentage>=60), 'First Class')
                  .when ((df_percent.percentage<60) & (df_percent.percentage>=50), 'Second Class')
                  .when ((df_percent.percentage<50) & (df_percent.percentage>=40), 'Third Class')
                  .when ((df_percent.percentage<40), 'Fail').alias('Result')
                )

display(df_final)


Id,Name,percentage,Result
1,Steve,95.0,Distinction
2,David,65.0,First Class
3,John,40.0,Third Class
5,Helen,45.0,Third Class
